In [2]:
import cv2
from typing import Tuple,List
from math import sin,cos,pi
from skimage.draw import line as line_points
import numpy as np
from numpy.linalg import norm
from PIL import Image

radian = pi/180.0 
sensorAngle = 250
sensorResolution = 2

In [3]:
'''
Takes the starting point of a line, its slope angle, and a length, and returns the ending point.
'''
def endLine(x1:int,y1:int,r:float,theta:float ):
    x2=int(x1+r*cos(theta*radian))
    y2=int(y1+r*sin(theta*radian))
    return x2 , y2    

'''
Draws a line on the image starting from a given point with a specified length, angle, color, and thickness.
'''
def drawLineOnImage(img:cv2.Mat,startPoint: Tuple[int,int],length: float,angle : float,color:Tuple[int,int,int],thickness: int):
    x1,y1=startPoint
    x2,y2=endLine(x1,y1,length,angle)
    return cv2.line(img,startPoint,(x2,y2),color,thickness)

'''
Simulates casting a ray from a given point in a specified direction until it hits an obstacle (pixel with color different from black).
Returns the endpoint of the ray.
'''
def castRay(img:cv2.Mat,startPoint: Tuple[int,int],maxLength: int,angle: float,color: Tuple[int,int,int]):

    x1,y1=startPoint
    x2,y2=endLine(x1,y1,maxLength,angle)

    # Get the coordinates of all points on the line
    linePoints = list(zip(*line_points(x1,y1,x2,y2)))
    endPoint = None
    for x,y in linePoints:
        endPoint = (x,y)
        # if the difference between current pixel and black 
        if norm(img[y,x] - np.array((0,0,0))) == 0 :
           
            endPoint = (x,y)
            break
    img = cv2.line(img,(x1,y1),endPoint,color,thickness=1)
    return endPoint

'''
Draws a robot on the map at a specified position (x, y) with a given orientation angle
'''
def drawRobot(map, x, y, theta):
    # Draw the filled circle representing the robot
    img = cv2.circle(map,center=(x, y),radius=10,color=(0,0,255),thickness=-1)

    # Draw the border of the circle representing the robot
    img = cv2.circle(img,center=(x, y),radius=10,color=(6,6,6),thickness=2)


    # Draw a line representing the direction of the robot
    img = drawLineOnImage(img,(x, y),length=20,angle=theta,color=(6,6,6),thickness=2)

    return img


In [4]:
#req1
def requirement1(imagePath: str , x: int, y: int, theta: int):
     
    # read the map 
    img=cv2.imread(imagePath,cv2.IMREAD_COLOR)
    # write the position of the robot in txt file
    file = open("measurements.txt", "w")
    file.write("Robot position: " + "x: " + str(x) + " y: " + str(y) + " theta: " + str(theta))

    # draw the robot on the map 
    img = drawRobot(img, x, y, theta)

     # create array to save the distance at each angle
    distances = np.zeros(int(250/2)+1)
    endpoints = []

    index = 0

    # write meaeurements in txt file
    file.write("\n\nMeasurements: ") 
    file.write("\nAngle\tEndpoint\tDistance")

    for angle in range(theta- int(250/ 2),theta+ int(250/2) + 1,2):
        endpoint = castRay(img,startPoint= (x,y),maxLength= int(12 *100/4), angle= angle,color= (0,255,0))
        # calculate the distance between the robot and the endpoint
        distance = norm(np.array((x,y)) - np.array(endpoint))

        # save the distance in the array
        distances[index] = min(1200/4, distance)
        endpoints.append(endpoint)

        # write meaeurements in txt file
        file.write("\n" + str(angle) + "\t\t\t" + str(endpoint)+"\t\t\t"+str(distances[index]))
        # increase the index
        index += 1



    file.close()

    cv2.imwrite('req1.jpg', img)

    return distances , np.array(endpoints)  
    


In [5]:
#req2
def requirement2(imagePath ,distance,sigma  ):
    
    # read the map
    map_ = cv2.imread(imagePath, cv2.IMREAD_GRAYSCALE)
    
    binary_image = cv2.threshold(map_, 127, 255, cv2.THRESH_BINARY)[1]
     # Perform the distance transform to get the closest obstacle
    d= cv2.distanceTransform(binary_image, cv2.DIST_L2, 0)

    d = np.array(d)
    # get the likelyhood from gayssian equation
    likelihood_field = 1/np.sqrt(2*pi*sigma) * np.exp(-0.5*((d)/sigma)**2)
   

    max_likelihood = np.max(likelihood_field)
   
    likelihood_field = likelihood_field/max_likelihood

  
    cv2.imwrite( f"likelihood field at sigma ={sigma}.png",likelihood_field*255)

   
 

    # create an array to save the probabilities of shape(num of rows , num of column)
    
    propbability_map = np.zeros(shape=(len(map_),len(map_[0])))
    # print(likelihood_field.shape[0])
    # print(likelihood_field.shape[1])
    
    for y in range (0 , likelihood_field.shape[0] , 1):#rows means y the height of the map
        for x in range (0 , likelihood_field.shape[1] ,1 ):#columns means x the widtht of the map
            arr=[]
            for angle in range (0,360 , 1):
                
                prob=1
                x2,y2= endLine(x1=x,y1=y,r=distance,theta=angle)
                if x2 < 0 or x2 >= likelihood_field.shape[1] or y2 < 0 or y2 >= likelihood_field.shape[0]:
                    prob *= 0.0000000000000001
                else:
                   prob*=likelihood_field[y2][x2]
                   
                arr.append(prob) 
                      
            propbability_map[y][x] =np.max(arr)  
               
    
    cv2.imwrite(f"probability map at length={distance*4/100} m , sigma ={sigma}.png",propbability_map*255) 



    max_index = np.unravel_index(np.argmax(propbability_map, axis=None), propbability_map.shape)

    # print the value of the point with the highest probability and its index
    print("The highest probability is: " ,(propbability_map[max_index[0]][max_index[1]]))
    print("The highest probability position is : ", max_index[1],max_index[0])  


  


    
   

    

    
    

In [6]:
img_path = "Map.jpg"
distances , endPonts = requirement1(img_path,350,185,200)
# distance = 0.2m , sigma =1,5,20
requirement2(img_path,0.2*100/4, 1)
requirement2(img_path,0.2*100/4, 5)
requirement2(img_path,0.2*100/4, 20)

# distance = 0.5m , sigma =1,5,20
requirement2(img_path,0.5*100/4, 1)
requirement2(img_path,0.5*100/4, 5)
requirement2(img_path,0.5*100/4, 20)

# distance = 0.8m , sigma =1,5,20
requirement2(img_path,0.8*100/4, 1)
requirement2(img_path,0.8*100/4, 5)
requirement2(img_path,0.8*100/4, 20)



# distance = 1m , sigma =1,5,20
requirement2(img_path,1*100/4, 1)
requirement2(img_path,1*100/4, 5)
requirement2(img_path,1*100/4, 20)

# distance = 2m , sigma =1,5,20
requirement2(img_path,2*100/4, 1)
requirement2(img_path,2*100/4, 5)
requirement2(img_path,2*100/4, 20)


# distance = 3m , sigma =1,5,20
requirement2(img_path,3*100/4, 1)
requirement2(img_path,3*100/4, 5)
requirement2(img_path,3*100/4, 20)

# # distance = 4m , sigma =1,5,20
# requirement2(img_path,4*100/4, 1)
# requirement2(img_path,4*100/4, 5)
# requirement2(img_path,4*100/4, 20)

# # distance = 5m , sigma =1,5,20
# requirement2(img_path,5*100/4, 1)
# requirement2(img_path,5*100/4, 5)
# requirement2(img_path,5*100/4, 20)


# # distance = 6m , sigma =1,5,20
# requirement2(img_path,6*100/4, 1)
# requirement2(img_path,6*100/4, 5)
# requirement2(img_path,6*100/4, 20)

# # distance = 12m , sigma =1,5,20
# requirement2(img_path,12*100/4, 1)
# requirement2(img_path,12*100/4, 5)
# requirement2(img_path,12*100/4, 20)
